In [1]:
!curl -LOJ https://github.com/alexeygrigorev/large-datasets/releases/download/wasps-bees/bees-wasps.h5

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 85.5M  100 85.5M    0     0   128M      0 --:--:-- --:--:-- --:--:--  245M


### Question 1
Now convert this model from Keras to TF-Lite format.

What's the size of the converted model?
- 21 Mb
- **43 Mb**
- 80 Mb
- 164 Mb

In [4]:
!ls -l bees-wasps.h5

-rw-r--r-- 1 jovyan jovyan 89753864 Nov 28 21:11 bees-wasps.h5


In [5]:
!pip install pillow

In [ ]:
!pip install tensorflow

In [8]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.lite as tflite
import numpy as np

2023-11-28 21:15:07.876087: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-28 21:15:07.919864: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-28 21:15:07.919901: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-28 21:15:07.921047: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-28 21:15:07.929454: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-28 21:15:07.930209: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [9]:
model = keras.models.load_model('bees-wasps.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('bee-wasps.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpeog4yiup/assets


INFO:tensorflow:Assets written to: /tmp/tmpeog4yiup/assets
2023-11-28 21:15:30.373349: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-28 21:15:30.373388: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-11-28 21:15:30.373981: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpeog4yiup
2023-11-28 21:15:30.374976: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-11-28 21:15:30.374996: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpeog4yiup
2023-11-28 21:15:30.376933: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2023-11-28 21:15:30.377707: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-11-28 21:15:30.521183: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

In [11]:
!ls -l bee-wasps.tflite

-rw-r--r-- 1 jovyan jovyan 44866200 Nov 28 21:15 bee-wasps.tflite


### Question 2
To be able to use this model, we need to know the index of the input and the index of the output.

What's the output index for this model?
- 3
- 7
- **13**
- 24

In [12]:
interpreter = tflite.Interpreter(model_path='bee-wasps.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']
print(input_index)
print(output_index)

0
13


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


### Question 3
Now we need to turn the image into numpy array and pre-process it.

Tip: Check the previous homework. What was the pre-processing we did there?

After the pre-processing, what's the value in the first pixel, the R channel?

- 0.3450980
- 0.5450980
- 0.7450980
- **0.9450980**

In [13]:
from io import BytesIO
from urllib import request
from PIL import Image

In [14]:
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

- Previous hw8 size:150,150

In [16]:
img = download_image('https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg')
img = prepare_image(img, target_size=(150, 150))
img

KeyboardInterrupt: 

In [ ]:
img_array = np.array(img, dtype='float32')
img_array = np.array([img_array])
img_array = img_array / 255.0
img_array[0,0,0,0]

0.94509804

### Question 4
Now let's apply this model to this image. What's the output of the model?

- 0.258
- 0.458
- **0.658**
- 0.858

In [ ]:
interpreter.set_tensor(input_index,img_array)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [ ]:
preds

array([[0.6592137]], dtype=float32)